In [ ]:
from dputils.scrape import Scraper, Tag

In [ ]:
url = 'https://www.flipkart.com/motorola-edge-50-pro-5g-68w-charger-black-beauty-256-gb/p/itmb3f6625c5c896?pid=MOBGXFXYDG8YRZU4&lid=LSTMOBGXFXYDG8YRZU4SAX9WW&marketplace=FLIPKART&store=tyy%2F4io&srno=b_1_1&otracker=browse&fm=organic&iid=ace38f80-151a-41b0-a079-b620b88b8ec9.MOBGXFXYDG8YRZU4.SEARCH&ppt=hp&ppn=homepage&ssid=vj43l0oe680000001721542596100'
s1 = Scraper(url)

In [ ]:
heading= Tag('h1', cls='_6EBuvT')
price = Tag(cls='Nx9bqj CxhGGd')
desc = Tag(cls='U+9u4y')
review = Tag(cls='RcXBOT')
results = s1.get_data_from_page(heading=heading, price=price, desc=desc, review=review)


multiple items from a single page

In [ ]:
items = Tag(cls='cPHDOP col-12-12')
url = "https://www.flipkart.com/tyy/4io/~cs-33irmtdz82/pr?sid=tyy%2C4io&collection-tab-name=Moto+Edge+50+pro+5g&param=279892&ctx=eyJjYXJkQ29udGV4dCI6eyJhdHRyaWJ1dGVzIjp7InZhbHVlQ2FsbG91dCI6eyJtdWx0aVZhbHVlZEF0dHJpYnV0ZSI6eyJrZXkiOiJ2YWx1ZUNhbGxvdXQiLCJpbmZlcmVuY2VUeXBlIjoiVkFMVUVfQ0FMTE9VVCIsInZhbHVlcyI6WyJGcm9tIOKCuTI3LDk5OSoiXSwidmFsdWVUeXBlIjoiTVVMVElfVkFMVUVEIn19LCJ0aXRsZSI6eyJtdWx0aVZhbHVlZEF0dHJpYnV0ZSI6eyJrZXkiOiJ0aXRsZSIsImluZmVyZW5jZVR5cGUiOiJUSVRMRSIsInZhbHVlcyI6WyJtb3Rvcm9sYSBlZGdlIDUwIFBybyJdLCJ2YWx1ZVR5cGUiOiJNVUxUSV9WQUxVRUQifX0sImhlcm9QaWQiOnsic2luZ2xlVmFsdWVBdHRyaWJ1dGUiOnsia2V5IjoiaGVyb1BpZCIsImluZmVyZW5jZVR5cGUiOiJQSUQiLCJ2YWx1ZSI6Ik1PQkdYRlhZTVRLQU1QVFMiLCJ2YWx1ZVR5cGUiOiJTSU5HTEVfVkFMVUVEIn19fX19"
s2 = Scraper(url)
results = s2.get_repeating_date_from_page(
    target = Tag('body'),
    items = items,
    title = Tag(cls='KzDlHZ'),
    price = Tag(cls='Nx9bqj _4b5DiR'),
    ratings = Tag('span', cls='Wphh3N')
)

In [ ]:
import pandas as pd
pd.DataFrame(results)

In [18]:
items = Tag(cls='s-result-item')
title = Tag('span', cls='a-size-base-plus a-color-base a-text-normal')
url ='https://www.amazon.in/s?k=shoes&crid=2OAOAMZS2EVJY&sprefix=shoe%2Caps%2C306&ref=nb_sb_noss_1'
s3 = Scraper(url)
results = s3.get_repeating_date_from_page(
    target = Tag('body'),
    items = items,
    title = title,
    brand = Tag('h2', cls='a-size-mini s-line-clamp-1')
)
pd.DataFrame(results).dropna()

,title,brand
2,Flash Casual Sneakers Riding Shoes for Men | S...,Campus
3,North Plus Sports Running Walking Gym Shoes fo...,Campus
4,Men's Atlantis Running Shoes,Campus
5,"Maxico Men's Running, Jogging, Gym, Sports, an...",Campus
6,Men's Dazzler Sneakers,Puma
...,...,...
70,Mens Ree Fusion M Sneaker,Reebok
71,Mens Ree Fusion M Sneaker,Reebok
72,Mens Energy Runner Lp Running Shoes,Reebok
73,Mens Boston Runner M Running Shoe,Reebok


In [20]:
start = 1
term = 'shoes'
all_data = []
while True:
    url = f"https://www.amazon.in/s?k={term}&page={start}&crid=2OAOAMZS2EVJY&qid=1721544045&sprefix=shoe%2Caps%2C306&ref=sr_pg_2"
    sc = Scraper(url)
    results = sc.get_repeating_date_from_page(
        target = Tag('body'),
        items = items,
        title = title,
        brand = Tag('h2', cls='a-size-mini s-line-clamp-1')
    )
    if len(results) == 0:
        break
    all_data.extend(results)
    start += 1

df = pd.DataFrame(all_data).dropna()
df